In [44]:
import feedparser
from xml.etree.ElementTree import Element, tostring
from xml.dom.minidom import parseString

import requests
import datetime, re, os, csv, copy
from collections import OrderedDict


In [38]:
RSS_FEED_PATH = r"https://rss.art19.com/wow-in-the-world"

In [39]:
def skip_me_fn(entry):
    ## returns true if we should skip this particular podcast entry
    if 'Two Whats' in entry.title:
        return True
    
    return False

In [40]:
def get_filtered_rss_feed(raw_feed_rss):
    raw_rss_dct = feedparser.parse(raw_feed_rss)

    ## filtered rss is exactly same as existing one to start out
    filtered_rss_dct = copy.deepcopy(raw_rss_dct)

    ## then we need to rebuild entries to exclude certain ones
    filtered_rss_dct["entries"] = []
    for entry in raw_rss_dct["entries"]:
        should_skip = ('itunes_episode' not in entry) or (skip_me_fn(entry))
        if should_skip: continue

        filtered_rss_dct["entries"].append(entry)

    return filtered_rss_dct

In [45]:
def dict_to_xml(tag, d):
    elem = Element(tag)
    if isinstance(d, dict):
        for key, val in d.items():
            child = dict_to_xml(key, val)
            elem.append(child)
    elif isinstance(d, list):
        for item in d:
            child = dict_to_xml(tag[:-1], item)  # Assuming plural tag for list
            elem.append(child)
    else:
        elem.text = str(d)
    return elem

In [58]:
def feedparser_to_rss(feed):
    rss = Element('rss', version='2.0')
    channel = dict_to_xml('channel', feed['feed'])
    rss.append(channel)
    
    items = []
    for entry in feed['entries']:
        item = dict_to_xml('item', entry)
        items.append(item)
    
    for item in items:
        channel.append(item)
    
    # Convert to a pretty XML string
    raw_xml = tostring(rss, 'utf-8')
    parsed_xml = parseString(raw_xml)
    pretty_xml = parsed_xml.toprettyxml(indent="\t")
    
    return pretty_xml

In [59]:
filtered_rss_dct = get_filtered_rss_feed(RSS_FEED_PATH)

In [60]:
filtered_rss_xml = feedparser_to_rss(filtered_rss_dct)

In [62]:
open(os.path.join(os.getcwd(), "test.xml"), 'w').write(filtered_rss_xml)

2379339

In [ ]:
open(os.path.join(os.getcwd(), "test.xml"), 'w').write(filtered_rss_xml)